## Лабораторная работа №3 
### DBScan кластеризация
### Экономика городов

In [1]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
%matplotlib inline

import os
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

In [2]:
df = pd.read_csv("Econom_Cities_data.csv", sep=";", decimal=',', index_col='City')
print(df)

                Work  Price  Salary
City                               
Amsterdam       1714   65.6    49.0
Athens          1792   53.8    30.4
Bogota          2152   37.9    11.5
Bombay          2052   30.3     5.3
Brussels        1708   73.8    50.5
Buenos_Aires    1971   56.1    12.5
Cairo          -9999   37.1 -9999.0
Caracas         2041   61.0    10.9
Chicago         1924   73.9    61.9
Copenhagen      1717   91.3    62.9
Dublin          1759   76.0    41.4
Dusseldorf      1693   78.5    60.2
Frankfurt       1650   74.5    60.4
Geneva          1880   95.9    90.3
Helsinki        1667  113.6    66.6
Hong_Kong       2375   63.8    27.8
Houston         1978   71.9    46.3
Jakarta        -9999   43.6 -9999.0
Johannesburg    1945   51.1    24.0
Kuala_Lumpur    2167   43.5     9.9
Lagos           1786   45.2     2.7
Lisbon          1742   56.2    18.8
London          1737   84.2    46.2
Los_Angeles     2068   79.8    65.2
Luxembourg      1768   71.1    71.1
Madrid          1710   93.8 

Проверим корректность данных. Можно заметить что в Cairo и Jakarta отрицательные Work и Price. Исключим их из выборки.

In [3]:
df = df.drop(index=["Jakarta","Cairo"])

Стандартизируем данные, поскольку данные из солбца Work значительно отличаются от данных в столбцах Salary и Price 

In [4]:
from sklearn import preprocessing
stand_1 = preprocessing.StandardScaler()
stand_1.fit(df)
X = stand_1.transform(df)
X = pd.DataFrame(X, index=df.index, columns=df.columns)
X.head()

,Work,Price,Salary
City,,,
Amsterdam,-0.962166,-0.212712,0.386095
Athens,-0.509827,-0.770489,-0.373488
Bogota,1.577891,-1.522069,-1.145323
Bombay,0.997969,-1.881316,-1.398517
Brussels,-0.996961,0.174896,0.447351


#### Подбираем параметры. Первый шаг.

In [5]:
dbscan_1 = DBSCAN(eps=1, metric='euclidean', min_samples=5)

dbscan_1.fit(X)
dbscan_1.labels_

array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1, -1, -1,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, -1,  0,  0,
        0,  0,  0,  0, -1,  0, -1,  0, -1,  0,  0, -1], dtype=int64)

Таблица частот

In [6]:
unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  8]
 [ 0 38]]


Получили только два кластера. Данные в них слишком отличаются поэтому попробуем уменьшить эпсилон

In [7]:
dbscan_1 = DBSCAN(eps=0.75, metric='euclidean', min_samples=5)
dbscan_1.fit(X)
dbscan_1.labels_

array([ 0,  1,  1,  1,  0,  1,  1,  0,  0,  0,  0,  0, -1, -1, -1,  0,  1,
        1,  1,  1,  0, -1,  0,  0,  1,  1,  0,  0,  1,  0,  1, -1,  1,  0,
        1,  1,  1,  1, -1,  0, -1,  1, -1,  0,  0, -1], dtype=int64)

Таблица частот

In [8]:
unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  9]
 [ 0 18]
 [ 1 19]]


Уменьшим min_samples пока не увеличится число кластеров

In [9]:
dbscan_1 = DBSCAN(eps=0.75, metric='euclidean', min_samples=2)
dbscan_1.fit(X)
dbscan_1.labels_
unique, counts = np.unique(dbscan_1.labels_, return_counts=True)
print(np.asarray((unique, counts)).T)

[[-1  4]
 [ 0 19]
 [ 1 19]
 [ 2  2]
 [ 3  2]]


Информация о последовательном слиянии кластеров

In [10]:
link = linkage(df, 'ward', 'euclidean')

Информация о кластерах dbscan добавляется в таблицу данных в виде столбца

In [11]:
df['dbscan_1'] = dbscan_1.labels_
print(df)

                Work  Price  Salary  dbscan_1
City                                         
Amsterdam       1714   65.6    49.0         0
Athens          1792   53.8    30.4         1
Bogota          2152   37.9    11.5         1
Bombay          2052   30.3     5.3         1
Brussels        1708   73.8    50.5         0
Buenos_Aires    1971   56.1    12.5         1
Caracas         2041   61.0    10.9         1
Chicago         1924   73.9    61.9         0
Copenhagen      1717   91.3    62.9         0
Dublin          1759   76.0    41.4         0
Dusseldorf      1693   78.5    60.2         0
Frankfurt       1650   74.5    60.4         0
Geneva          1880   95.9    90.3         2
Helsinki        1667  113.6    66.6         3
Hong_Kong       2375   63.8    27.8        -1
Houston         1978   71.9    46.3         0
Johannesburg    1945   51.1    24.0         1
Kuala_Lumpur    2167   43.5     9.9         1
Lagos           1786   45.2     2.7         1
Lisbon          1742   56.2    18.

In [12]:
tab = pd.crosstab(dbscan_1.labels_, df['dbscan_1'])
print(tab)

dbscan_1  -1   0   1   2   3
row_0                       
-1         4   0   0   0   0
 0         0  19   0   0   0
 1         0   0  19   0   0
 2         0   0   0   2   0
 3         0   0   0   0   2


In [13]:
df['dbscan_1'].sort_values()

City
Tokyo            -1
Taipei           -1
Stockholm        -1
Hong_Kong        -1
Amsterdam         0
Toronto           0
Sydney            0
Paris             0
New_York          0
Montreal          0
Milan             0
Madrid            0
Vienna            0
Los_Angeles       0
London            0
Luxembourg        0
Houston           0
Chicago           0
Copenhagen        0
Brussels          0
Frankfurt         0
Dusseldorf        0
Dublin            0
Johannesburg      1
Seoul             1
Buenos_Aires      1
San_Paulo         1
Bombay            1
Tel_Aviv          1
Bogota            1
Athens            1
Singpore          1
Rio_de_Janeiro    1
Nicosia           1
Panama            1
Nairobi           1
Mexico_City       1
Manila            1
Lisbon            1
Lagos             1
Kuala_Lumpur      1
Caracas           1
Geneva            2
Zurich            2
Oslo              3
Helsinki          3
Name: dbscan_1, dtype: int64

Усредним значения для анализа

In [14]:
df.groupby('dbscan_1').mean()

,Work,Price,Salary
dbscan_1,,,
-1,2051.250000,93.600000,42.375000
0,1792.000000,77.526316,55.157895
1,1959.210526,50.115789,14.789474
2,1874.000000,97.950000,95.150000
3,1625.000000,114.550000,65.150000


In [15]:
print ('Медиана work = ',df['Work'].median())
print ('Медиана price = ',df['Price'].median())
print ('Медиана salary = ',df['Salary'].median())

Медиана work =  1849.0
Медиана price =  70.94999999999999
Медиана salary =  43.65


### в -1 группе
С высоким рабочим средневзвешенным, высокими ценами и средней зарплатой
В эту группу входят Токио, Тайпей, Гонк-Конг и Стокгольм

### в 0 группе
города  с низким рабочим средневзвешенным, низкими показателями цен и средней зарплатой
В эту группу входят крупные города Европы, Северной Америки, Сидней и Монреаль
### в 1 группе
Города с высоким рабочим средневзвешенным, низкими показателями цен и заработных плат
В эту группу входят города Южной Америки, Юго-Восточной Азии и Африки,Кипра, города Сеул и Лагос 

### в 2 города
Со средним рабочим средневзвешенным, высокими ценами и зарплатой
В эту группу входят крупные города Швейцарии: Женева и Цюрих

### в 3 группе
Низкое рабочее средневзвешенное, высокие показатели цен и заработной платы. 
В эту группу входят города Скандинавии: Осло и Хельсинки